In [1]:
# KNN. 

In [2]:
from __future__ import division
from mnist import MNIST
from sklearn import tree
import numpy as np
from sklearn.metrics import classification_report,log_loss,accuracy_score,roc_auc_score
from time import time
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
import re
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix


In [3]:
# Loading the data
mndata = MNIST('./')
images_train, labels_train = mndata.load_training()
images_test, labels_test = mndata.load_testing()
labels_test = np.array(labels_test)

images_train = np.array(images_train)
labels_train = np.array(labels_train)


In [4]:
from sklearn.neighbors import KNeighborsClassifier

In [5]:
excel_file_name = 'KNN' + str(datetime.now().strftime('%Y%m%d%H%M%S')) + '.xlsx'
excel_writer = pd.ExcelWriter(excel_file_name, engine='openpyxl')
df = pd.DataFrame(columns=['Test'])
df.to_excel(excel_writer, sheet_name='demo_sheet')

excel_writer.save()

/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


In [ ]:
def classifaction_report_to_dataframe(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-5]:
        row = {}
        row_data = line.split('      ')
        if len(row_data)>4:
            row['class'] = row_data[1]
            row['precision'] = float(row_data[2])
            row['recall'] = float(row_data[3])
            row['f1_score'] = float(row_data[4])
            row['support'] = float(row_data[5])
            report_data.append(row)
    for line in lines[-4:-1]:
        row = {}
        p = re.compile("[a-z]+ [a-z]+")
        m = p.search(line)  
        row['class'] = m.group(0)
        p = re.compile("\d+.?\d+")
        row_data = p.findall(line)
        row['precision'] = float(row_data[0])
        row['recall'] = float(row_data[1])
        row['f1_score'] = float(row_data[2])
        row['support'] = float(row_data[3])
        report_data.append(row)
        
        
    dataframe = pd.DataFrame.from_dict(report_data)
    return dataframe


In [ ]:
''' 
# Code for tesing purpose 
images_train = images_train[0:100]
labels_train = labels_train[0:100]
images_test = images_test[0:100]
labels_test = labels_test[0:100]
'''

' \n# Code for tesing purpose \nimages_train = images_train[0:100]\nlabels_train = labels_train[0:100]\nimages_test = images_test[0:100]\nlabels_test = labels_test[0:100]\n'

In [ ]:
neighbours = [1,10,20,25,30,50,100,200,300,500]
# neighbours = [1]

df = pd.DataFrame(columns=['Number of neighbours', 'Accuracy','Normalized Accuracy', 
                 'Time_taken_train','Time_taken_test',
                 'Negative Log loss','Macro_auc','Micro_auc'])
for n in neighbours:
    neigh = KNeighborsClassifier(n_neighbors=n)
    
    start = time()
    neigh.fit(images_train, labels_train) 
    train_time = time() - start
    
    start = time()
    predictions = neigh.predict(images_test)
    test_time = time() - start
   
    accuracy = accuracy_score(predictions,labels_test,normalize=False)
    normalized_accuracy = accuracy_score(predictions,labels_test,normalize=True)
    log_loss_prediction = neigh.predict_proba(images_test)
    log_loss_val = log_loss(labels_test,log_loss_prediction) 
    classfication_repo = classification_report(labels_test, predictions)
    
    
    
    labels_train_one_hot=label_binarize(labels_test,classes=[0,1,2,3,4,5,6,7,8,9])
    predictions_one_hot=label_binarize(predictions,classes=[0,1,2,3,4,5,6,7,8,9])
    micro_auc=roc_auc_score(labels_train_one_hot,predictions_one_hot,average="micro")
    macro_auc=roc_auc_score(labels_train_one_hot,predictions_one_hot,average="macro")
    
    cm = confusion_matrix(labels_test,predictions)
    cm_df = pd.DataFrame(data=cm)
    
    
    
    df_data = [n,accuracy,normalized_accuracy,train_time,test_time,log_loss_val,macro_auc,micro_auc]
    df.loc[len(df)] = df_data
    
    book = load_workbook(excel_file_name)
    with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
        excel_writer.book = book
        df_classification_report = classifaction_report_to_dataframe(classfication_repo)
        df_classification_report.to_excel(excel_writer, sheet_name='knn_cr' + '_'+ str(n))
        
        cm_df.to_excel(excel_writer, sheet_name='knn_cr' + '_'+ str(n),
                       startrow=len(df_classification_report)+5)

        excel_writer.save()    
    
book = load_workbook(excel_file_name)
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
    excel_writer.book = book
    df.to_excel(excel_writer, sheet_name='knn')
    excel_writer.save()    

/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


In [ ]:
''' 
# Code for tesing purpose 
images_train = images_train[0:100]
labels_train = labels_train[0:100]
images_test = images_test[0:100]
labels_test = labels_test[0:100]
'''

In [ ]:
distances = [1,2,3]
neighbours = [1,2,5]
# distances = [1]
# neighbours = [1]
df = pd.DataFrame(columns=['Neighbours','Distance used', 'Accuracy','Normalized Accuracy', 
                 'Time_taken_train','Time_taken_test',
                 'Negative Log loss','Macro_auc','Micro_auc'])
for n in neighbours:
    for d in distances:
        neigh = KNeighborsClassifier(n_neighbors=5,p=d)

        start = time()
        neigh.fit(images_train, labels_train) 
        train_time = time() - start

        start = time()
        predictions = neigh.predict(images_test)
        test_time = time() - start

        accuracy = accuracy_score(predictions,labels_test,normalize=False)
        normalized_accuracy = accuracy_score(predictions,labels_test,normalize=True)
        log_loss_prediction = neigh.predict_proba(images_test)
        log_loss_val = log_loss(labels_test,log_loss_prediction) 
        classfication_repo = classification_report(labels_test, predictions)



        labels_train_one_hot=label_binarize(labels_test,classes=[0,1,2,3,4,5,6,7,8,9])
        predictions_one_hot=label_binarize(predictions,classes=[0,1,2,3,4,5,6,7,8,9])
        micro_auc=roc_auc_score(labels_train_one_hot,predictions_one_hot,average="micro")
        macro_auc=roc_auc_score(labels_train_one_hot,predictions_one_hot,average="macro")


        cm = confusion_matrix(labels_test,predictions)
        cm_df = pd.DataFrame(data=cm)

        df_data = [n,d,accuracy,normalized_accuracy,train_time,test_time,log_loss_val,macro_auc,micro_auc]
        df.loc[len(df)] = df_data

        book = load_workbook(excel_file_name)
        with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
            excel_writer.book = book
            df_classification_report = classifaction_report_to_dataframe(classfication_repo)
            df_classification_report.to_excel(excel_writer, sheet_name='knn_cr_d' + '_'+ str(n))
            
            cm_df.to_excel(excel_writer, sheet_name='knn_cr_d' + '_'+ str(n),
                       startrow=len(df_classification_report)+5)

            excel_writer.save()    
    
book = load_workbook(excel_file_name)
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
    excel_writer.book = book
    df.to_excel(excel_writer, sheet_name='knn_distances')
    excel_writer.save()    